In [1]:
import os
import openpyxl

In [2]:
path_list = []

for curDir, dirs, files in os.walk("新聞"):
    #         print('=='*20)
    #         print('現在のフォルダ: ' + curDir)
    #         print('内包するフォルダ: ', end='')
    #         print(dirs)
    #         print('内包するファイル: ', end='')
    #         print(files)
    path = curDir + '\\'
    for file in files:
        path += file
        #             print('ファイルパス: ' + path)
        path_list.append(path)
        path = curDir + '\\'

In [3]:
headlines = []  # 格納用辞書

wb1 = openpyxl.load_workbook(path_list[0])
sheet = wb1['シート1']
# [社名、年、月、名前、見出し、ベクトル]
headlines = [(row[0].value, row[1].value, row[2].value, row[3].value, row[4].value) for row in sheet.rows]

wb2 = openpyxl.load_workbook(path_list[1])
sheet = wb2['シート1']
headlines2 = [(row[0].value, row[1].value, row[2].value, row[3].value, row[4].value) for row in sheet.rows]

In [4]:
headlines += headlines2

# 分かち書き

In [5]:
import MeCab as mc
import mojimoji
m = mc.Tagger("-Ochasen")

In [6]:
def mecab_tokenizer(text: str):
    """
    テキストを分かち書きするメソッド
    :param text: 分割したいテキスト
    :return: 分割後のテキスト
    """

    node = m.parseToNode(text)
    word_list = list()
    while node:
        if node.surface != "":
            res = node.feature.split(",")
            word_type = res[0]
            if word_type in ['名詞', "動詞", "形容詞", "副詞"]:  # 名詞, 動詞, 形容詞, 副詞のみを抽出
                basic_word = res[6]
                if basic_word != "*":
                    word_list.append(basic_word)
                else:
                    word_list.append('[UNK]')  # 未知語の場合は[UNK]トークンに置き換え
        node = node.next
        if node is None:
            break
    return word_list

In [7]:
def clean_text(text: str):
    """
    テキストの正規化
    :param text: 正規化するテキスト
    :return: 正規化後のテキスト
    """
    text = mojimoji.han_to_zen(text, digit=False, ascii=False)  # 半角文字を全角文字に統一(数字, 英語以外)
    text = mojimoji.zen_to_han(text, kana=False)  # 全角文字を半角文字に統一(かな以外)
    text = text.lower()  # 小文字に統一
    return text

In [8]:
from gensim.models import KeyedVectors
import numpy as np

file_name = "jawiki.all_vectors.200d-002.txt"

model = KeyedVectors.load_word2vec_format(file_name)

In [9]:
def vectorize_word(text: str):
    """
    元word_analysis()
    ベクトル化を行うメソッド
    :param text: ベクトル化するテキスト
    :return: ベクトル化したテキスト(array型)
    """

    text = clean_text(text)

    V = list()  # 文章のベクトル(200次元)を格納

    # 文章の対して, 文章中の単語のベクトルの平均を求める処理を行う
    word_list = mecab_tokenizer(text)
    v = np.array([0.0] * 200)
    word_num = 0
    for word in word_list:
        try:
            v += np.array(model[word])
        except KeyError as error:
            continue
        except ValueError as verror:
            continue
        word_num += 1

    try:
        v = v / word_num
    except ZeroDivisionError as e:
        print(f'ZeroDivisionError: {e}')

    return v

def cos_sim(v1, v2):
    """
    cos類似度を計算
    :param v1: word2vecによりベクトル化したテキスト1(ndarray型)
    :param v2: word2vecによりベクトル化したテキスト2(ndarray型)
    :return: 二つのテキストのcos類似度
    """
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [10]:
def get_news_dict(filename: str):
    """
    エクセルから見出しのタイトル(識別番号？)とテキストを取り出しベクトル化して辞書として返す
    :param filename: excelファイル名
    :return: ひと月分の辞書<br>
    一つの要素に{(会社名、年、月、名前):(見出し, ベクトル)} -> の形で返す
    """
    wb = openpyxl.load_workbook(filename)
    sheet = wb['シート1']
    dict = {}  # 格納用辞書
    for row in sheet.rows:

        if row[4].value is not None:
            # vector = vectorize_word(row[4].value)  # 見出しをベクトル化
            words = mecab_tokenizer(row[4].value)
            words = [word for word in words if word != '[UNK]']
            dict[(row[0].value, row[1].value, row[2].value, row[3].value)] = (row[4].value, words)  # (会社名、年、月、名前)と(見出し、ベクトル)の組

    return dict

In [11]:
all_newspapers = [get_news_dict(path) for path in path_list]

In [13]:
all_newspapers

[{('こども大分新聞',
   '1932年',
   '10月',
   '2-y-2_01'): ('んぐみ\n| day\n三人組の隨一 ジ一堅い信念を貫いた。\nBEATION FLOR ANATUREHOU\n||||', ['ぐみ',
    '三',
    '人組',
    '一',
    '堅い',
    '信念',
    '貫く']),
  ('こども大分新聞',
   '1932年',
   '10月',
   '2-y-2_02'): ('四戒を守って 健やか&伸びる 專科に運動に大いに養\n日田男子校の巻', ['四',
    '守る',
    '健やか',
    '伸びる',
    '運動',
    '大いに',
    '養',
    '日田',
    '男子校',
    '巻']),
  ('こども大分新聞', '1932年', '10月', '2-y-2_03'): ('少年 少 女 講 座 メートル法の起源',
   ['少年', '女', '講', '座', 'メートル法', '起源']),
  ('こども大分新聞',
   '1932年',
   '10月',
   '2-y-9_01'): ('10WH\n||||||||RUNWHIEF\n電器具と 附屬品のいろく\n文明國の小國民は\n是非知って置きませう', ['電',
    '器具',
    '附',
    'いる',
    'く',
    '文明',
    '國',
    '小國',
    '民',
    '是非',
    '知る',
    '置く']),
  ('こども大分新聞', '1932年', '10月', '2-y-9_04'): ('hateway.\nいろいろ\n電燈線の',
   ['いろいろ', '電']),
  ('こども大分新聞', '1932年', '10月', '2-y-4_01'): ('日本は殊に 霞が招はい それは気温を 氣壁の配置から」',
   ['日本', '殊に', '霞', 'それ', '気温', '配置']),
  ('こども大分新聞', '1932年', '10月', '2-a-8_01'): ('の 發 明', ['明']),
  ('こども大分新聞', '1932年',

In [14]:
headline_list = [(name[0], name[1], name[2], name[3], headline[0], headline[1]) for news in all_newspapers \
                for name, headline in news.items()]

In [15]:
headline_list

[('こども大分新聞',
  '1932年',
  '10月',
  '2-y-2_01',
  'んぐみ\n| day\n三人組の隨一 ジ一堅い信念を貫いた。\nBEATION FLOR ANATUREHOU\n||||',
  ['ぐみ', '三', '人組', '一', '堅い', '信念', '貫く']),
 ('こども大分新聞',
  '1932年',
  '10月',
  '2-y-2_02',
  '四戒を守って 健やか&伸びる 專科に運動に大いに養\n日田男子校の巻',
  ['四', '守る', '健やか', '伸びる', '運動', '大いに', '養', '日田', '男子校', '巻']),
 ('こども大分新聞',
  '1932年',
  '10月',
  '2-y-2_03',
  '少年 少 女 講 座 メートル法の起源',
  ['少年', '女', '講', '座', 'メートル法', '起源']),
 ('こども大分新聞',
  '1932年',
  '10月',
  '2-y-9_01',
  '10WH\n||||||||RUNWHIEF\n電器具と 附屬品のいろく\n文明國の小國民は\n是非知って置きませう',
  ['電', '器具', '附', 'いる', 'く', '文明', '國', '小國', '民', '是非', '知る', '置く']),
 ('こども大分新聞',
  '1932年',
  '10月',
  '2-y-9_04',
  'hateway.\nいろいろ\n電燈線の',
  ['いろいろ', '電']),
 ('こども大分新聞',
  '1932年',
  '10月',
  '2-y-4_01',
  '日本は殊に 霞が招はい それは気温を 氣壁の配置から」',
  ['日本', '殊に', '霞', 'それ', '気温', '配置']),
 ('こども大分新聞', '1932年', '10月', '2-a-8_01', 'の 發 明', ['明']),
 ('こども大分新聞',
  '1932年',
  '10月',
  '2-a-8_03',
  'でんわ\nくった\n電話機の組立',
  ['でんわ', 'くる', '電話機', '組立']),
 ('こども大分新聞',
  '1932年',

In [16]:
import pickle

In [17]:
with open('keyword.pickle', mode='wb') as f:
    pickle.dump(headline_list, f)

In [ ]:
serach_index = {} # 検索用インデックス